In [7]:
import pandas as pd
import numpy as np

df= pd.read_csv('modeling_dataframe.csv')
print(df.shape)

(55454, 144)


In [8]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
df.drop(columns=['air_yards_for_model','yards_after_catch','yards_after_catch_missing'], inplace=True)
X = df.drop(columns=['success_outcome'])
y = df['success_outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")  

Mean Squared Error: 0.0554011445309516
R^2 Score: 0.10799959082276422


In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

X = df.drop(columns=['success_outcome'])
y = df['success_outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")  

Mean Squared Error: 0.05790493383360443
R^2 Score: 0.06768668571279568


In [12]:
# ==========================================
# Optuna tuning for regression (RF + GB) using RMSE (stable multiprocessing)
# ==========================================

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, TimeSeriesSplit, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor

import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner

# ====== Required: your data ======
y_col = 'success_outcome'
assert 'df' in globals(), "You must have a DataFrame named `df` in memory."
assert y_col in df.columns, f"'{y_col}' not found in df.columns"

# ====== Config ======
RANDOM_STATE = 42
RF_TRIALS = 60
GB_TRIALS = 80

# Use RMSE as the metric
SCORING = 'neg_root_mean_squared_error'

# Avoid BrokenProcessPool: run CV in main process
CV_N_JOBS = 1
ESTIMATOR_N_JOBS = -1

# -------------------
# Data prep
# -------------------
model_df = df.copy()
X_cols = [c for c in model_df.columns if c != y_col]

date_col = 'game_date' if 'game_date' in model_df.columns else None
if date_col:
    model_df[date_col] = pd.to_datetime(model_df[date_col], errors='coerce')
    model_df = model_df.dropna(subset=[date_col]).sort_values(date_col)

X_all = model_df[X_cols]
y_all = model_df[y_col]
mask = y_all.notna()
X_all = X_all.loc[mask]
y_all = y_all.loc[mask]

# -------------------
# CV setup
# -------------------
if date_col:
    cv = TimeSeriesSplit(n_splits=5)
    print("[CV] Using TimeSeriesSplit (n_splits=5).")
else:
    cv = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
    print("[CV] Using KFold (n_splits=5).")

# -------------------
# Utility: cross-validated RMSE (positive)
# -------------------
def cv_score(estimator, X, y, cv, scoring):
    # cross_val_score with 'neg_root_mean_squared_error' returns negative values; invert sign to get RMSE.
    scores = cross_val_score(estimator, X, y, cv=cv, scoring=scoring, n_jobs=CV_N_JOBS)
    rmse = -np.mean(scores)
    return float(rmse)

# -------------------
# 1) RandomForestRegressor tuning
# -------------------
def tune_random_forest(X, y, cv, n_trials=60, random_state=RANDOM_STATE):
    def objective(trial):
        n_estimators = trial.suggest_int("n_estimators", 100, 600)
        max_depth = trial.suggest_int("max_depth", 6, 30)
        min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
        min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)
        max_features = trial.suggest_float("max_features", 0.2, 1.0)

        est = RandomForestRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            max_features=max_features,
            n_jobs=ESTIMATOR_N_JOBS,
            random_state=random_state,
        )
        pipe = make_pipeline(SimpleImputer(strategy='median'), est)
        return cv_score(pipe, X, y, cv, SCORING)  # returns positive RMSE

    study = optuna.create_study(
        direction="minimize",  # minimize RMSE
        sampler=TPESampler(seed=random_state),
        pruner=MedianPruner()
    )
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

    print("\n[RF] Best CV RMSE: {:.5f}".format(study.best_value))
    print("[RF] Best params:", study.best_params)

    best_params = study.best_params
    best_est = RandomForestRegressor(**best_params, n_jobs=ESTIMATOR_N_JOBS, random_state=random_state)
    best_model = make_pipeline(SimpleImputer(strategy='median'), best_est).fit(X, y)
    return study, best_model

# -------------------
# 2) HistGradientBoostingRegressor tuning
# -------------------
def tune_hist_gb(X, y, cv, n_trials=80, random_state=RANDOM_STATE):
    def objective(trial):
        params = {
            "learning_rate": trial.suggest_float("learning_rate", 0.02, 0.3, log=True),
            "max_depth": trial.suggest_int("max_depth", 2, 12),
            "max_leaf_nodes": trial.suggest_int("max_leaf_nodes", 15, 63),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 20, 256),
            "l2_regularization": trial.suggest_float("l2_regularization", 0.0, 1.0),
            "max_iter": trial.suggest_int("max_iter", 200, 800),
        }
        est = HistGradientBoostingRegressor(
            **params,
            random_state=random_state,
            early_stopping=True,
            validation_fraction=0.1,
        )
        return cv_score(est, X, y, cv, SCORING)  # returns positive RMSE

    study = optuna.create_study(
        direction="minimize",  # minimize RMSE
        sampler=TPESampler(seed=random_state),
        pruner=MedianPruner()
    )
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

    print("\n[GB] Best CV RMSE: {:.5f}".format(study.best_value))
    print("[GB] Best params:", study.best_params)

    best_params = study.best_params
    best_model = HistGradientBoostingRegressor(
        **best_params,
        random_state=random_state,
        early_stopping=True,
        validation_fraction=0.1,
    ).fit(X, y)
    return study, best_model

# -------------------
# Run tuning
# -------------------
if __name__ == "__main__":
    print("\n=== Tuning RandomForestRegressor ===")
    rf_study, best_rf = tune_random_forest(X_all, y_all, cv, n_trials=RF_TRIALS)

    print("\n=== Tuning HistGradientBoostingRegressor ===")
    gb_study, best_gb = tune_hist_gb(X_all, y_all, cv, n_trials=GB_TRIALS)

    # Final CV evaluation (report RMSE mean ± std)
    print("\n[Final 5-fold CV] Tuned model performance (RMSE):")
    rf_scores = cross_val_score(best_rf, X_all, y_all, cv=cv, scoring=SCORING, n_jobs=CV_N_JOBS)
    gb_scores = cross_val_score(best_gb, X_all, y_all, cv=cv, scoring=SCORING, n_jobs=CV_N_JOBS)

    rf_rmse = -rf_scores  # per-fold RMSE
    gb_rmse = -gb_scores

    print(f"RandomForest RMSE: {rf_rmse.mean():.5f} ± {rf_rmse.std():.5f}")
    print(f"HistGradientBoosting RMSE: {gb_rmse.mean():.5f} ± {gb_rmse.std():.5f}")

    print("\n=== Best Params Summary ===")
    print("[RandomForest]", rf_study.best_params)
    print("[HistGradientBoosting]", gb_study.best_params)

    print("\nTuned models are available as: best_rf, best_gb")


[I 2025-11-23 20:38:23,062] A new study created in memory with name: no-name-53181d61-f395-4be3-ac99-f3408badd0b6


[CV] Using KFold (n_splits=5).

=== Tuning RandomForestRegressor ===


Best trial: 0. Best value: 0.241016:   2%|▏         | 1/60 [01:02<1:01:50, 62.89s/it]

[I 2025-11-23 20:39:25,949] Trial 0 finished with value: 0.24101597163113037 and parameters: {'n_estimators': 287, 'max_depth': 29, 'min_samples_split': 15, 'min_samples_leaf': 6, 'max_features': 0.3248149123539492}. Best is trial 0 with value: 0.24101597163113037.


Best trial: 0. Best value: 0.241016:   3%|▎         | 2/60 [01:28<39:32, 40.91s/it]  

...

[I 2025-11-24 10:11:21,957] Trial 77 finished with value: 0.23981930263598117 and parameters: {'learning_rate': 0.02480830860372825, 'max_depth': 6, 'max_leaf_nodes': 41, 'min_samples_leaf': 225, 'l2_regularization': 0.6291745819917922, 'max_iter': 681}. Best is trial 71 with value: 0.23948145938135493.


Best trial: 71. Best value: 0.239481:  99%|█████████▉| 79/80 [5:33:27<00:14, 14.21s/it]

[I 2025-11-24 10:11:32,247] Trial 78 finished with value: 0.23967132719649217 and parameters: {'learning_rate': 0.05062514807503121, 'max_depth': 10, 'max_leaf_nodes': 37, 'min_samples_leaf': 182, 'l2_regularization': 0.7633652849195196, 'max_iter': 707}. Best is trial 71 with value: 0.23948145938135493.


Best trial: 71. Best value: 0.239481: 100%|██████████| 80/80 [5:33:44<00:00, 250.31s/it]


[I 2025-11-24 10:11:49,920] Trial 79 finished with value: 0.23973557543895896 and parameters: {'learning_rate': 0.02691798455386636, 'max_depth': 11, 'max_leaf_nodes': 51, 'min_samples_leaf': 231, 'l2_regularization': 0.7915289964279778, 'max_iter': 746}. Best is trial 71 with value: 0.23948145938135493.

[GB] Best CV RMSE: 0.23948
[GB] Best params: {'learning_rate': 0.04089599581936438, 'max_depth': 12, 'max_leaf_nodes': 35, 'min_samples_leaf': 222, 'l2_regularization': 0.6835980422000931, 'max_iter': 741}

[Final 5-fold CV] Tuned model performance (RMSE):
RandomForest RMSE: 0.24065 ± 0.00254
HistGradientBoosting RMSE: 0.23948 ± 0.00260

=== Best Params Summary ===
[RandomForest] {'n_estimators': 588, 'max_depth': 28, 'min_samples_split': 15, 'min_samples_leaf': 8, 'max_features': 0.23629126287997582}
[HistGradientBoosting] {'learning_rate': 0.04089599581936438, 'max_depth': 12, 'max_leaf_nodes': 35, 'min_samples_leaf': 222, 'l2_regularization': 0.6835980422000931, 'max_iter': 741}

T

In [16]:
df['success_outcome'].describe()

count    55454.000000
mean         0.180014
std          0.254279
min          0.000000
25%          0.000000
50%          0.089552
75%          0.239130
max          1.000000
Name: success_outcome, dtype: float64

In [ ]:
# # ==========================================
# # Optuna tuning for regression (RF + GB)
# # ==========================================

# import warnings
# warnings.filterwarnings("ignore")

# import numpy as np
# import pandas as pd

# from sklearn.model_selection import KFold, TimeSeriesSplit, cross_val_score
# from sklearn.pipeline import make_pipeline
# from sklearn.impute import SimpleImputer
# from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor

# import optuna
# from optuna.samplers import TPESampler
# from optuna.pruners import MedianPruner

# # ====== Required: your data ======
# y_col = 'success_outcome'
# assert 'df' in globals(), "You must have a DataFrame named `df` in memory."
# assert y_col in df.columns, f"'{y_col}' not found in df.columns"

# # ====== Config ======
# RANDOM_STATE = 42
# RF_TRIALS = 60
# GB_TRIALS = 80
# SCORING = 'r2'

# # -------------------
# # Data prep
# # -------------------
# model_df = df.copy()
# X_cols = [c for c in model_df.columns if c != y_col]

# date_col = 'game_date' if 'game_date' in model_df.columns else None
# if date_col:
#     model_df[date_col] = pd.to_datetime(model_df[date_col], errors='coerce')
#     model_df = model_df.dropna(subset=[date_col]).sort_values(date_col)

# X_all = model_df[X_cols]
# y_all = model_df[y_col]
# mask = y_all.notna()
# X_all = X_all.loc[mask]
# y_all = y_all.loc[mask]

# # -------------------
# # CV setup
# # -------------------
# if date_col:
#     cv = TimeSeriesSplit(n_splits=5)
#     print("[CV] Using TimeSeriesSplit (n_splits=5).")
# else:
#     cv = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
#     print("[CV] Using KFold (n_splits=5).")

# # -------------------
# # Utility: cross-validated score
# # -------------------
# def cv_score(estimator, X, y, cv, scoring):
#     scores = cross_val_score(estimator, X, y, cv=cv, scoring=scoring, n_jobs=-1)
#     return float(np.mean(scores))

# # -------------------
# # 1) RandomForestRegressor tuning
# # -------------------
# def tune_random_forest(X, y, cv, n_trials=60, random_state=RANDOM_STATE):
#     def objective(trial):
#         n_estimators = trial.suggest_int("n_estimators", 100, 600)
#         max_depth = trial.suggest_int("max_depth", 6, 30)
#         min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
#         min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)
#         max_features = trial.suggest_float("max_features", 0.2, 1.0)

#         est = RandomForestRegressor(
#             n_estimators=n_estimators,
#             max_depth=max_depth,
#             min_samples_split=min_samples_split,
#             min_samples_leaf=min_samples_leaf,
#             max_features=max_features,
#             n_jobs=-1,
#             random_state=random_state,
#         )
#         pipe = make_pipeline(SimpleImputer(strategy='median'), est)
#         return cv_score(pipe, X, y, cv, SCORING)

#     study = optuna.create_study(direction="maximize", sampler=TPESampler(seed=random_state), pruner=MedianPruner())
#     study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

#     print("\n[RF] Best CV R²: {:.5f}".format(study.best_value))
#     print("[RF] Best params:", study.best_params)

#     best_params = study.best_params
#     best_est = RandomForestRegressor(**best_params, n_jobs=-1, random_state=random_state)
#     best_model = make_pipeline(SimpleImputer(strategy='median'), best_est).fit(X, y)
#     return study, best_model

# # -------------------
# # 2) HistGradientBoostingRegressor tuning
# # -------------------
# def tune_hist_gb(X, y, cv, n_trials=80, random_state=RANDOM_STATE):
#     def objective(trial):
#         params = {
#             "learning_rate": trial.suggest_float("learning_rate", 0.02, 0.3, log=True),
#             "max_depth": trial.suggest_int("max_depth", 2, 12),
#             "max_leaf_nodes": trial.suggest_int("max_leaf_nodes", 15, 63),
#             "min_samples_leaf": trial.suggest_int("min_samples_leaf", 20, 256),
#             "l2_regularization": trial.suggest_float("l2_regularization", 0.0, 1.0),
#             "max_iter": trial.suggest_int("max_iter", 200, 800),
#         }
#         est = HistGradientBoostingRegressor(
#             **params,
#             random_state=random_state,
#             early_stopping=True,
#             validation_fraction=0.1,
#         )
#         return cv_score(est, X, y, cv, SCORING)

#     study = optuna.create_study(direction="maximize", sampler=TPESampler(seed=random_state), pruner=MedianPruner())
#     study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

#     print("\n[GB] Best CV R²: {:.5f}".format(study.best_value))
#     print("[GB] Best params:", study.best_params)

#     best_params = study.best_params
#     best_model = HistGradientBoostingRegressor(
#         **best_params,
#         random_state=random_state,
#         early_stopping=True,
#         validation_fraction=0.1,
#     ).fit(X, y)
#     return study, best_model

# # -------------------
# # Run tuning
# # -------------------
# if __name__ == "__main__":
#     print("\n=== Tuning RandomForestRegressor ===")
#     rf_study, best_rf = tune_random_forest(X_all, y_all, cv, n_trials=RF_TRIALS)

#     print("\n=== Tuning HistGradientBoostingRegressor ===")
#     gb_study, best_gb = tune_hist_gb(X_all, y_all, cv, n_trials=GB_TRIALS)

#     # Final CV evaluation
#     print("\n[Final 5-fold CV] Tuned model performance:")
#     rf_cv = cross_val_score(best_rf, X_all, y_all, cv=cv, scoring=SCORING, n_jobs=-1)
#     gb_cv = cross_val_score(best_gb, X_all, y_all, cv=cv, scoring=SCORING, n_jobs=-1)

#     print(f"RandomForest R²: {rf_cv.mean():.5f} ± {rf_cv.std():.5f}")
#     print(f"HistGradientBoosting R²: {gb_cv.mean():.5f} ± {gb_cv.std():.5f}")

#     print("\n=== Best Params Summary ===")
#     print("[RandomForest]", rf_study.best_params)
#     print("[HistGradientBoosting]", gb_study.best_params)

#     print("\nTuned models are available as: best_rf, best_gb")


[I 2025-11-23 14:46:49,275] A new study created in memory with name: no-name-825a4c7b-108b-4e42-8bd7-cf9be8799feb


[CV] Using KFold (n_splits=5).

=== Tuning RandomForestRegressor ===


  0%|          | 0/60 [00:01<?, ?it/s]


[W 2025-11-23 14:46:50,739] Trial 0 failed with parameters: {'n_estimators': 287, 'max_depth': 29, 'min_samples_split': 15, 'min_samples_leaf': 6, 'max_features': 0.3248149123539492} because of the following error: BrokenProcessPool('A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.').
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/Users/johnburzawa/Library/Python/3.9/lib/python/site-packages/joblib/externals/loky/process_executor.py", line 453, in _process_worker
    call_item = call_queue.get(block=True, timeout=timeout)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
ModuleNotFoundError: No module named 'pandas.core.indexes.numeric'
"""

The above exception was the direct cause of the following exception:

Traceback (most recent cal

BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

In [ ]:
'''[I 2025-11-15 18:51:41,517] A new study created in memory with name: no-name-81268d34-fa41-493b-b654-2764393dd3b4
[CV] Using KFold (n_splits=5).

=== Tuning RandomForestRegressor ===
Best trial: 0. Best value: 0.709715:   2%|▏         | 1/60 [00:39<38:46, 39.44s/it]
[I 2025-11-15 18:52:20,952] Trial 0 finished with value: 0.7097145587552562 and parameters: {'n_estimators': 287, 'max_depth': 29, 'min_samples_split': 15, 'min_samples_leaf': 6, 'max_features': 0.3248149123539492}. Best is trial 0 with value: 0.7097145587552562.
Best trial: 0. Best value: 0.709715:   3%|▎         | 2/60 [01:01<28:24, 29.39s/it]
[I 2025-11-15 18:52:43,315] Trial 1 finished with value: 0.6520257070058123 and parameters: {'n_estimators': 178, 'max_depth': 7, 'min_samples_split': 18, 'min_samples_leaf': 7, 'max_features': 0.7664580622368364}. Best is trial 0 with value: 0.7097145587552562.
Best trial: 2. Best value: 0.710481:   5%|▌         | 3/60 [01:19<22:46, 23.98s/it]
[I 2025-11-15 18:53:00,851] Trial 2 finished with value: 0.7104805493214379 and parameters: {'n_estimators': 110, 'max_depth': 30, 'min_samples_split': 17, 'min_samples_leaf': 3, 'max_features': 0.3454599737656805}. Best is trial 2 with value: 0.7104805493214379.
Best trial: 2. Best value: 0.710481:   7%|▋         | 4/60 [01:44<22:44, 24.37s/it]
[I 2025-11-15 18:53:25,812] Trial 3 finished with value: 0.7053167367627727 and parameters: {'n_estimators': 191, 'max_depth': 13, 'min_samples_split': 11, 'min_samples_leaf': 5, 'max_features': 0.43298331215843355}. Best is trial 2 with value: 0.7104805493214379.
Best trial: 2. Best value: 0.710481:   8%|▊         | 5/60 [02:32<30:21, 33.11s/it]
[I 2025-11-15 18:54:14,436] Trial 4 finished with value: 0.6826120115082748 and parameters: {'n_estimators': 406, 'max_depth': 9, 'min_samples_split': 7, 'min_samples_leaf': 4, 'max_features': 0.5648559873736287}. Best is trial 2 with value: 0.7104805493214379.
Best trial: 2. Best value: 0.710481:  10%|█         | 6/60 [03:03<28:59, 32.21s/it]
[I 2025-11-15 18:54:44,902] Trial 5 finished with value: 0.6613622661573705 and parameters: {'n_estimators': 493, 'max_depth': 10, 'min_samples_split': 11, 'min_samples_leaf': 6, 'max_features': 0.2371603301759982}. Best is trial 2 with value: 0.7104805493214379.
Best trial: 2. Best value: 0.710481:  12%|█▏        | 7/60 [04:32<44:50, 50.76s/it]
[I 2025-11-15 18:56:13,854] Trial 6 finished with value: 0.6877418091224428 and parameters: {'n_estimators': 404, 'max_depth': 10, 'min_samples_split': 3, 'min_samples_leaf': 10, 'max_features': 0.9725056264596474}. Best is trial 2 with value: 0.7104805493214379.
Best trial: 2. Best value: 0.710481:  13%|█▎        | 8/60 [05:59<54:02, 62.36s/it]
[I 2025-11-15 18:57:41,035] Trial 7 finished with value: 0.7061190931267339 and parameters: {'n_estimators': 505, 'max_depth': 13, 'min_samples_split': 3, 'min_samples_leaf': 7, 'max_features': 0.5521219949916811}. Best is trial 2 with value: 0.7104805493214379.
Best trial: 2. Best value: 0.710481:  15%|█▌        | 9/60 [06:26<43:38, 51.34s/it]
[I 2025-11-15 18:58:08,156] Trial 8 finished with value: 0.7082299561873763 and parameters: {'n_estimators': 161, 'max_depth': 18, 'min_samples_split': 2, 'min_samples_leaf': 10, 'max_features': 0.40702398528001354}. Best is trial 2 with value: 0.7104805493214379.
Best trial: 2. Best value: 0.710481:  17%|█▋        | 10/60 [07:17<42:43, 51.27s/it]
[I 2025-11-15 18:58:59,269] Trial 9 finished with value: 0.7015593533844153 and parameters: {'n_estimators': 431, 'max_depth': 13, 'min_samples_split': 11, 'min_samples_leaf': 6, 'max_features': 0.3478835644204217}. Best is trial 2 with value: 0.7104805493214379.
Best trial: 2. Best value: 0.710481:  18%|█▊        | 11/60 [09:00<54:50, 67.14s/it]
[I 2025-11-15 19:00:42,401] Trial 10 finished with value: 0.7080424544782041 and parameters: {'n_estimators': 293, 'max_depth': 29, 'min_samples_split': 20, 'min_samples_leaf': 1, 'max_features': 0.7214932406225427}. Best is trial 2 with value: 0.7104805493214379.
Best trial: 2. Best value: 0.710481:  20%|██        | 12/60 [09:34<45:26, 56.80s/it]
[I 2025-11-15 19:01:15,559] Trial 11 finished with value: 0.7032140615866498 and parameters: {'n_estimators': 288, 'max_depth': 30, 'min_samples_split': 15, 'min_samples_leaf': 2, 'max_features': 0.2060245106292583}. Best is trial 2 with value: 0.7104805493214379.
Best trial: 2. Best value: 0.710481:  22%|██▏       | 13/60 [09:51<35:13, 44.96s/it]
[I 2025-11-15 19:01:33,274] Trial 12 finished with value: 0.7100873395953332 and parameters: {'n_estimators': 106, 'max_depth': 25, 'min_samples_split': 16, 'min_samples_leaf': 3, 'max_features': 0.3261610808221125}. Best is trial 2 with value: 0.7104805493214379.
Best trial: 13. Best value: 0.712132:  23%|██▎       | 14/60 [10:15<29:35, 38.59s/it]
[I 2025-11-15 19:01:57,149] Trial 13 finished with value: 0.7121323450581714 and parameters: {'n_estimators': 105, 'max_depth': 24, 'min_samples_split': 15, 'min_samples_leaf': 3, 'max_features': 0.4853229247739137}. Best is trial 13 with value: 0.7121323450581714.
Best trial: 14. Best value: 0.712848:  25%|██▌       | 15/60 [10:45<27:03, 36.08s/it]
[I 2025-11-15 19:02:27,413] Trial 14 finished with value: 0.7128480258847854 and parameters: {'n_estimators': 130, 'max_depth': 23, 'min_samples_split': 13, 'min_samples_leaf': 3, 'max_features': 0.47810674439726614}. Best is trial 14 with value: 0.7128480258847854.
Best trial: 14. Best value: 0.712848:  27%|██▋       | 16/60 [13:22<52:58, 72.25s/it]
[I 2025-11-15 19:05:03,645] Trial 15 finished with value: 0.7126760152143948 and parameters: {'n_estimators': 583, 'max_depth': 22, 'min_samples_split': 8, 'min_samples_leaf': 1, 'max_features': 0.49247626486277407}. Best is trial 14 with value: 0.7128480258847854.
Best trial: 14. Best value: 0.712848:  28%|██▊       | 17/60 [56:40<9:56:10, 831.88s/it]
[I 2025-11-15 19:48:22,108] Trial 16 finished with value: 0.7108881906768314 and parameters: {'n_estimators': 600, 'max_depth': 21, 'min_samples_split': 7, 'min_samples_leaf': 1, 'max_features': 0.6490922148236045}. Best is trial 14 with value: 0.7128480258847854.
Best trial: 14. Best value: 0.712848:  30%|███       | 18/60 [2:26:17<25:38:13, 2197.47s/it]
[I 2025-11-15 21:17:58,547] Trial 17 finished with value: 0.7099523996346525 and parameters: {'n_estimators': 600, 'max_depth': 19, 'min_samples_split': 7, 'min_samples_leaf': 2, 'max_features': 0.8331235173260678}. Best is trial 14 with value: 0.7128480258847854.
Best trial: 14. Best value: 0.712848:  32%|███▏      | 19/60 [3:05:00<25:27:30, 2235.37s/it]
[I 2025-11-15 21:56:42,217] Trial 18 finished with value: 0.7115974491857157 and parameters: {'n_estimators': 232, 'max_depth': 24, 'min_samples_split': 9, 'min_samples_leaf': 1, 'max_features': 0.5281357569359176}. Best is trial 14 with value: 0.7128480258847854.
Best trial: 14. Best value: 0.712848:  33%|███▎      | 20/60 [6:00:28<52:30:07, 4725.19s/it]
[I 2025-11-16 00:52:10,363] Trial 19 finished with value: 0.7122510869414285 and parameters: {'n_estimators': 343, 'max_depth': 20, 'min_samples_split': 12, 'min_samples_leaf': 4, 'max_features': 0.6546684521201547}. Best is trial 14 with value: 0.7128480258847854.
Best trial: 14. Best value: 0.712848:  35%|███▌      | 21/60 [7:34:48<54:13:43, 5005.72s/it]
[I 2025-11-16 02:26:30,130] Trial 20 finished with value: 0.7122850593344174 and parameters: {'n_estimators': 515, 'max_depth': 17, 'min_samples_split': 13, 'min_samples_leaf': 2, 'max_features': 0.4569282758642466}. Best is trial 14 with value: 0.7128480258847854.
Best trial: 14. Best value: 0.712848:  37%|███▋      | 22/60 [8:45:52<50:29:15, 4783.03s/it]
[I 2025-11-16 03:37:33,845] Trial 21 finished with value: 0.7114433028174789 and parameters: {'n_estimators': 529, 'max_depth': 16, 'min_samples_split': 13, 'min_samples_leaf': 2, 'max_features': 0.4678097527672477}. Best is trial 14 with value: 0.7128480258847854.
Best trial: 22. Best value: 0.713123:  38%|███▊      | 23/60 [13:21:22<85:23:20, 8308.12s/it]
[I 2025-11-16 08:13:03,944] Trial 22 finished with value: 0.7131225356614704 and parameters: {'n_estimators': 548, 'max_depth': 22, 'min_samples_split': 9, 'min_samples_leaf': 2, 'max_features': 0.5988834090435163}. Best is trial 22 with value: 0.7131225356614704.
Best trial: 22. Best value: 0.713123:  40%|████      | 24/60 [14:36:12<71:37:26, 7162.39s/it]
[I 2025-11-16 09:27:53,712] Trial 23 finished with value: 0.7130413279505592 and parameters: {'n_estimators': 552, 'max_depth': 22, 'min_samples_split': 9, 'min_samples_leaf': 4, 'max_features': 0.6017287968495193}. Best is trial 22 with value: 0.7131225356614704.
Best trial: 22. Best value: 0.713123:  42%|████▏     | 25/60 [14:38:17<49:06:28, 5051.09s/it]
[I 2025-11-16 09:29:59,368] Trial 24 finished with value: 0.7129877096806322 and parameters: {'n_estimators': 455, 'max_depth': 26, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 0.6404555566402557}. Best is trial 22 with value: 0.7131225356614704.
Best trial: 22. Best value: 0.713123:  43%|████▎     | 26/60 [14:40:17<33:43:53, 3571.56s/it]
[I 2025-11-16 09:31:59,168] Trial 25 finished with value: 0.712884736733223 and parameters: {'n_estimators': 454, 'max_depth': 26, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 0.6296249365380793}. Best is trial 22 with value: 0.7131225356614704.
Best trial: 22. Best value: 0.713123:  45%|████▌     | 27/60 [14:42:52<23:20:36, 2546.55s/it]
[I 2025-11-16 09:34:34,242] Trial 26 finished with value: 0.7117252401479607 and parameters: {'n_estimators': 543, 'max_depth': 27, 'min_samples_split': 9, 'min_samples_leaf': 5, 'max_features': 0.7104479694998717}. Best is trial 22 with value: 0.7131225356614704.
Best trial: 22. Best value: 0.713123:  47%|████▋     | 28/60 [14:45:18<16:13:57, 1826.16s/it]
[I 2025-11-16 09:36:59,596] Trial 27 finished with value: 0.7073530042846681 and parameters: {'n_estimators': 469, 'max_depth': 27, 'min_samples_split': 5, 'min_samples_leaf': 8, 'max_features': 0.8426281909906951}. Best is trial 22 with value: 0.7131225356614704.
Best trial: 28. Best value: 0.71365:  48%|████▊     | 29/60 [15:03:00<13:45:08, 1597.06s/it] 
[I 2025-11-16 09:54:42,133] Trial 28 finished with value: 0.7136500401882513 and parameters: {'n_estimators': 558, 'max_depth': 21, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 0.5851935778327936}. Best is trial 28 with value: 0.7136500401882513.
Best trial: 28. Best value: 0.71365:  50%|█████     | 30/60 [15:05:08<9:38:09, 1156.33s/it] 
[I 2025-11-16 09:56:50,104] Trial 29 finished with value: 0.7129613548642004 and parameters: {'n_estimators': 552, 'max_depth': 21, 'min_samples_split': 9, 'min_samples_leaf': 5, 'max_features': 0.5830604024084933}. Best is trial 28 with value: 0.7136500401882513.
Best trial: 28. Best value: 0.71365:  52%|█████▏    | 31/60 [15:15:45<8:03:33, 1000.46s/it]
[I 2025-11-16 10:07:26,889] Trial 30 finished with value: 0.7104135569695899 and parameters: {'n_estimators': 576, 'max_depth': 15, 'min_samples_split': 6, 'min_samples_leaf': 4, 'max_features': 0.6944895262066334}. Best is trial 28 with value: 0.7136500401882513.
Best trial: 28. Best value: 0.71365:  53%|█████▎    | 32/60 [15:17:42<5:43:10, 735.37s/it] 
[I 2025-11-16 10:09:23,717] Trial 31 finished with value: 0.7131437703272802 and parameters: {'n_estimators': 473, 'max_depth': 22, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 0.612164240785154}. Best is trial 28 with value: 0.7136500401882513.
Best trial: 28. Best value: 0.71365:  55%|█████▌    | 33/60 [15:20:39<4:15:36, 568.02s/it]
[I 2025-11-16 10:12:21,265] Trial 32 finished with value: 0.711222742471352 and parameters: {'n_estimators': 560, 'max_depth': 22, 'min_samples_split': 4, 'min_samples_leaf': 5, 'max_features': 0.773941700291415}. Best is trial 28 with value: 0.7136500401882513.
Best trial: 28. Best value: 0.71365:  57%|█████▋    | 34/60 [15:22:41<3:08:10, 434.27s/it]
[I 2025-11-16 10:14:23,425] Trial 33 finished with value: 0.7131441815793249 and parameters: {'n_estimators': 486, 'max_depth': 19, 'min_samples_split': 8, 'min_samples_leaf': 3, 'max_features': 0.5912000309547457}. Best is trial 28 with value: 0.7136500401882513.
Best trial: 28. Best value: 0.71365:  58%|█████▊    | 35/60 [15:24:15<2:18:20, 332.00s/it]
[I 2025-11-16 10:15:56,818] Trial 34 finished with value: 0.7134643952264926 and parameters: {'n_estimators': 483, 'max_depth': 19, 'min_samples_split': 6, 'min_samples_leaf': 3, 'max_features': 0.39801756253389375}. Best is trial 28 with value: 0.7136500401882513.
Best trial: 28. Best value: 0.71365:  60%|██████    | 36/60 [15:25:28<1:41:44, 254.36s/it]
[I 2025-11-16 10:17:10,018] Trial 35 finished with value: 0.7135923036505861 and parameters: {'n_estimators': 369, 'max_depth': 19, 'min_samples_split': 6, 'min_samples_leaf': 3, 'max_features': 0.4037591722117687}. Best is trial 28 with value: 0.7136500401882513.
Best trial: 28. Best value: 0.71365:  62%|██████▏   | 37/60 [15:26:38<1:16:19, 199.10s/it]
[I 2025-11-16 10:18:20,163] Trial 36 finished with value: 0.7134549065196925 and parameters: {'n_estimators': 372, 'max_depth': 19, 'min_samples_split': 6, 'min_samples_leaf': 3, 'max_features': 0.38934624679681107}. Best is trial 28 with value: 0.7136500401882513.
Best trial: 28. Best value: 0.71365:  63%|██████▎   | 38/60 [15:27:35<57:20, 156.39s/it]  
[I 2025-11-16 10:19:16,898] Trial 37 finished with value: 0.7098017093476789 and parameters: {'n_estimators': 373, 'max_depth': 15, 'min_samples_split': 2, 'min_samples_leaf': 3, 'max_features': 0.3898149126946108}. Best is trial 28 with value: 0.7136500401882513.
Best trial: 28. Best value: 0.71365:  65%|██████▌   | 39/60 [15:28:17<42:45, 122.17s/it]
[I 2025-11-16 10:19:59,214] Trial 38 finished with value: 0.7090069284074072 and parameters: {'n_estimators': 322, 'max_depth': 18, 'min_samples_split': 4, 'min_samples_leaf': 3, 'max_features': 0.2838229734374577}. Best is trial 28 with value: 0.7136500401882513.
Best trial: 28. Best value: 0.71365:  67%|██████▋   | 40/60 [15:29:10<33:48, 101.42s/it]
[I 2025-11-16 10:20:52,212] Trial 39 finished with value: 0.7071411897184694 and parameters: {'n_estimators': 389, 'max_depth': 20, 'min_samples_split': 7, 'min_samples_leaf': 7, 'max_features': 0.2813678694833013}. Best is trial 28 with value: 0.7136500401882513.
Best trial: 28. Best value: 0.71365:  68%|██████▊   | 41/60 [15:29:27<24:04, 76.04s/it] 
[I 2025-11-16 10:21:09,056] Trial 40 finished with value: 0.61307495969032 and parameters: {'n_estimators': 246, 'max_depth': 6, 'min_samples_split': 3, 'min_samples_leaf': 5, 'max_features': 0.3832624227926524}. Best is trial 28 with value: 0.7136500401882513.
Best trial: 28. Best value: 0.71365:  70%|███████   | 42/60 [15:31:07<24:55, 83.10s/it]
[I 2025-11-16 10:22:48,620] Trial 41 finished with value: 0.7128908261399188 and parameters: {'n_estimators': 434, 'max_depth': 17, 'min_samples_split': 8, 'min_samples_leaf': 3, 'max_features': 0.5237738317588156}. Best is trial 28 with value: 0.7136500401882513.
Best trial: 28. Best value: 0.71365:  72%|███████▏  | 43/60 [15:32:10<21:49, 77.06s/it]
[I 2025-11-16 10:23:51,579] Trial 42 finished with value: 0.7106535270659882 and parameters: {'n_estimators': 428, 'max_depth': 19, 'min_samples_split': 6, 'min_samples_leaf': 2, 'max_features': 0.2833278862980477}. Best is trial 28 with value: 0.7136500401882513.
Best trial: 43. Best value: 0.713714:  73%|███████▎  | 44/60 [15:33:48<22:15, 83.49s/it]
[I 2025-11-16 10:25:30,067] Trial 43 finished with value: 0.7137136947510042 and parameters: {'n_estimators': 486, 'max_depth': 19, 'min_samples_split': 6, 'min_samples_leaf': 3, 'max_features': 0.41473514762898145}. Best is trial 43 with value: 0.7137136947510042.
Best trial: 43. Best value: 0.713714:  75%|███████▌  | 45/60 [15:34:51<19:18, 77.26s/it]
[I 2025-11-16 10:26:32,805] Trial 44 finished with value: 0.7084103511909212 and parameters: {'n_estimators': 373, 'max_depth': 14, 'min_samples_split': 6, 'min_samples_leaf': 3, 'max_features': 0.42494993272322246}. Best is trial 43 with value: 0.7137136947510042.
Best trial: 43. Best value: 0.713714:  77%|███████▋  | 46/60 [15:35:44<16:22, 70.19s/it]
[I 2025-11-16 10:27:26,484] Trial 45 finished with value: 0.7104244435085368 and parameters: {'n_estimators': 333, 'max_depth': 17, 'min_samples_split': 4, 'min_samples_leaf': 4, 'max_features': 0.34893219925057534}. Best is trial 43 with value: 0.7137136947510042.
Best trial: 43. Best value: 0.713714:  78%|███████▊  | 47/60 [15:37:14<16:28, 76.04s/it]
[I 2025-11-16 10:28:56,182] Trial 46 finished with value: 0.7109164629535469 and parameters: {'n_estimators': 504, 'max_depth': 18, 'min_samples_split': 3, 'min_samples_leaf': 6, 'max_features': 0.371477384451421}. Best is trial 43 with value: 0.7137136947510042.
Best trial: 43. Best value: 0.713714:  80%|████████  | 48/60 [15:38:17<14:23, 71.95s/it]
[I 2025-11-16 10:29:58,600] Trial 47 finished with value: 0.69572696759038 and parameters: {'n_estimators': 413, 'max_depth': 11, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 0.4379586193361296}. Best is trial 43 with value: 0.7137136947510042.
Best trial: 43. Best value: 0.713714:  82%|████████▏ | 49/60 [15:39:10<12:09, 66.32s/it]
[I 2025-11-16 10:30:51,761] Trial 48 finished with value: 0.7119641087762532 and parameters: {'n_estimators': 305, 'max_depth': 20, 'min_samples_split': 6, 'min_samples_leaf': 3, 'max_features': 0.3176683458783478}. Best is trial 43 with value: 0.7137136947510042.
Best trial: 43. Best value: 0.713714:  83%|████████▎ | 50/60 [15:39:39<09:11, 55.16s/it]
[I 2025-11-16 10:31:20,896] Trial 49 finished with value: 0.7004703364260945 and parameters: {'n_estimators': 254, 'max_depth': 16, 'min_samples_split': 4, 'min_samples_leaf': 5, 'max_features': 0.24101107558630863}. Best is trial 43 with value: 0.7137136947510042.
Best trial: 43. Best value: 0.713714:  85%|████████▌ | 51/60 [15:41:15<10:07, 67.55s/it]
[I 2025-11-16 10:32:57,366] Trial 50 finished with value: 0.7135223390224746 and parameters: {'n_estimators': 377, 'max_depth': 19, 'min_samples_split': 7, 'min_samples_leaf': 4, 'max_features': 0.5361450576651556}. Best is trial 43 with value: 0.7137136947510042.
Best trial: 43. Best value: 0.713714:  87%|████████▋ | 52/60 [15:42:50<10:06, 75.76s/it]
[I 2025-11-16 10:34:32,260] Trial 51 finished with value: 0.7134809050389823 and parameters: {'n_estimators': 358, 'max_depth': 19, 'min_samples_split': 7, 'min_samples_leaf': 4, 'max_features': 0.5376486080310214}. Best is trial 43 with value: 0.7137136947510042.
Best trial: 43. Best value: 0.713714:  88%|████████▊ | 53/60 [15:44:42<10:05, 86.53s/it]
[I 2025-11-16 10:36:23,922] Trial 52 finished with value: 0.7136244752255562 and parameters: {'n_estimators': 392, 'max_depth': 21, 'min_samples_split': 7, 'min_samples_leaf': 4, 'max_features': 0.5413862241201347}. Best is trial 43 with value: 0.7137136947510042.
Best trial: 43. Best value: 0.713714:  90%|█████████ | 54/60 [15:46:20<09:00, 90.12s/it]
[I 2025-11-16 10:38:02,416] Trial 53 finished with value: 0.7135210510337322 and parameters: {'n_estimators': 358, 'max_depth': 21, 'min_samples_split': 7, 'min_samples_leaf': 4, 'max_features': 0.5439045869555371}. Best is trial 43 with value: 0.7137136947510042.
Best trial: 43. Best value: 0.713714:  92%|█████████▏| 55/60 [15:48:06<07:53, 94.75s/it]
[I 2025-11-16 10:39:47,964] Trial 54 finished with value: 0.7135327835500564 and parameters: {'n_estimators': 397, 'max_depth': 24, 'min_samples_split': 8, 'min_samples_leaf': 5, 'max_features': 0.508557827501612}. Best is trial 43 with value: 0.7137136947510042.
Best trial: 43. Best value: 0.713714:  93%|█████████▎| 56/60 [15:49:51<06:30, 97.71s/it]
[I 2025-11-16 10:41:32,577] Trial 55 finished with value: 0.7133817194188048 and parameters: {'n_estimators': 397, 'max_depth': 24, 'min_samples_split': 10, 'min_samples_leaf': 5, 'max_features': 0.5201462223615718}. Best is trial 43 with value: 0.7137136947510042.
Best trial: 43. Best value: 0.713714:  95%|█████████▌| 57/60 [15:51:33<04:56, 98.99s/it]
[I 2025-11-16 10:43:14,549] Trial 56 finished with value: 0.7127271503124153 and parameters: {'n_estimators': 422, 'max_depth': 23, 'min_samples_split': 8, 'min_samples_leaf': 6, 'max_features': 0.5061405799645439}. Best is trial 43 with value: 0.7137136947510042.
Best trial: 43. Best value: 0.713714:  97%|█████████▋| 58/60 [15:53:06<03:14, 97.40s/it]
[I 2025-11-16 10:44:48,236] Trial 57 finished with value: 0.7118152081932392 and parameters: {'n_estimators': 451, 'max_depth': 23, 'min_samples_split': 10, 'min_samples_leaf': 7, 'max_features': 0.44887906829654933}. Best is trial 43 with value: 0.7137136947510042.
Best trial: 43. Best value: 0.713714:  98%|█████████▊| 59/60 [15:54:26<01:32, 92.03s/it]
[I 2025-11-16 10:46:07,737] Trial 58 finished with value: 0.7112645572168608 and parameters: {'n_estimators': 309, 'max_depth': 25, 'min_samples_split': 20, 'min_samples_leaf': 5, 'max_features': 0.5728682617160024}. Best is trial 43 with value: 0.7137136947510042.
Best trial: 59. Best value: 0.71408: 100%|██████████| 60/60 [15:56:31<00:00, 956.52s/it]
[I 2025-11-16 10:48:12,627] Trial 59 finished with value: 0.7140803627639197 and parameters: {'n_estimators': 528, 'max_depth': 21, 'min_samples_split': 8, 'min_samples_leaf': 4, 'max_features': 0.4839384174656995}. Best is trial 59 with value: 0.7140803627639197.

[RF] Best CV R²: 0.71408
[RF] Best params: {'n_estimators': 528, 'max_depth': 21, 'min_samples_split': 8, 'min_samples_leaf': 4, 'max_features': 0.4839384174656995}
[I 2025-11-16 10:48:44,711] A new study created in memory with name: no-name-4c7d3fc5-e6a4-4f24-b233-5797b959e7d9

=== Tuning HistGradientBoostingRegressor ===
Best trial: 0. Best value: 0.734928:   1%|▏         | 1/80 [00:14<18:57, 14.40s/it]
[I 2025-11-16 10:48:59,111] Trial 0 finished with value: 0.7349282607532083 and parameters: {'learning_rate': 0.05514718587869894, 'max_depth': 12, 'max_leaf_nodes': 50, 'min_samples_leaf': 161, 'l2_regularization': 0.15601864044243652, 'max_iter': 293}. Best is trial 0 with value: 0.7349282607532083.
Best trial: 0. Best value: 0.734928:   2%|▎         | 2/80 [00:50<35:28, 27.29s/it]
[I 2025-11-16 10:49:35,422] Trial 1 finished with value: 0.7344129318879767 and parameters: {'learning_rate': 0.023406777359270506, 'max_depth': 11, 'max_leaf_nodes': 44, 'min_samples_leaf': 187, 'l2_regularization': 0.020584494295802447, 'max_iter': 782}. Best is trial 0 with value: 0.7349282607532083.
Best trial: 0. Best value: 0.734928:   4%|▍         | 3/80 [00:58<23:30, 18.31s/it]
[I 2025-11-16 10:49:43,052] Trial 2 finished with value: 0.7260375346523009 and parameters: {'learning_rate': 0.19057174434080473, 'max_depth': 4, 'max_leaf_nodes': 23, 'min_samples_leaf': 63, 'l2_regularization': 0.3042422429595377, 'max_iter': 515}. Best is trial 0 with value: 0.7349282607532083.
Best trial: 0. Best value: 0.734928:   5%|▌         | 4/80 [01:12<20:52, 16.48s/it]
[I 2025-11-16 10:49:56,727] Trial 3 finished with value: 0.7315549004991764 and parameters: {'learning_rate': 0.06442237870408891, 'max_depth': 5, 'max_leaf_nodes': 44, 'min_samples_leaf': 53, 'l2_regularization': 0.29214464853521815, 'max_iter': 420}. Best is trial 0 with value: 0.7349282607532083.
Best trial: 0. Best value: 0.734928:   6%|▋         | 5/80 [01:21<17:13, 13.78s/it]
[I 2025-11-16 10:50:05,730] Trial 4 finished with value: 0.7298857575063545 and parameters: {'learning_rate': 0.06877172495876586, 'max_depth': 10, 'max_leaf_nodes': 24, 'min_samples_leaf': 141, 'l2_regularization': 0.5924145688620425, 'max_iter': 227}. Best is trial 0 with value: 0.7349282607532083.
Best trial: 0. Best value: 0.734928:   8%|▊         | 6/80 [01:37<18:01, 14.62s/it]
[I 2025-11-16 10:50:21,975] Trial 5 finished with value: 0.7131974905601657 and parameters: {'learning_rate': 0.10364734587283783, 'max_depth': 3, 'max_leaf_nodes': 18, 'min_samples_leaf': 244, 'l2_regularization': 0.9656320330745594, 'max_iter': 685}. Best is trial 0 with value: 0.7349282607532083.
Best trial: 0. Best value: 0.734928:   9%|▉         | 7/80 [01:49<16:47, 13.79s/it]
[I 2025-11-16 10:50:34,069] Trial 6 finished with value: 0.7049161019000111 and parameters: {'learning_rate': 0.045633479761632394, 'max_depth': 3, 'max_leaf_nodes': 48, 'min_samples_leaf': 124, 'l2_regularization': 0.12203823484477883, 'max_iter': 497}. Best is trial 0 with value: 0.7349282607532083.
Best trial: 0. Best value: 0.734928:  10%|█         | 8/80 [02:10<19:32, 16.29s/it]
[I 2025-11-16 10:50:55,701] Trial 7 finished with value: 0.727742675769034 and parameters: {'learning_rate': 0.02195199700425886, 'max_depth': 12, 'max_leaf_nodes': 27, 'min_samples_leaf': 177, 'l2_regularization': 0.31171107608941095, 'max_iter': 512}. Best is trial 0 with value: 0.7349282607532083.
Best trial: 0. Best value: 0.734928:  11%|█▏        | 9/80 [02:26<18:52, 15.95s/it]
[I 2025-11-16 10:51:10,909] Trial 8 finished with value: 0.7218759732356587 and parameters: {'learning_rate': 0.08790451384972603, 'max_depth': 4, 'max_leaf_nodes': 62, 'min_samples_leaf': 203, 'l2_regularization': 0.9394989415641891, 'max_iter': 737}. Best is trial 0 with value: 0.7349282607532083.
Best trial: 0. Best value: 0.734928:  12%|█▎        | 10/80 [02:36<16:41, 14.30s/it]
[I 2025-11-16 10:51:21,525] Trial 9 finished with value: 0.7327621725722727 and parameters: {'learning_rate': 0.10097524940480988, 'max_depth': 12, 'max_leaf_nodes': 19, 'min_samples_leaf': 66, 'l2_regularization': 0.045227288910538066, 'max_iter': 395}. Best is trial 0 with value: 0.7349282607532083.
Best trial: 0. Best value: 0.734928:  14%|█▍        | 11/80 [02:41<13:12, 11.48s/it]
[I 2025-11-16 10:51:26,608] Trial 10 finished with value: 0.7278094909505362 and parameters: {'learning_rate': 0.25858112699745844, 'max_depth': 8, 'max_leaf_nodes': 59, 'min_samples_leaf': 109, 'l2_regularization': 0.6400885024675562, 'max_iter': 223}. Best is trial 0 with value: 0.7349282607532083.
Best trial: 0. Best value: 0.734928:  15%|█▌        | 12/80 [03:10<18:45, 16.55s/it]
[I 2025-11-16 10:51:54,750] Trial 11 finished with value: 0.730755340691059 and parameters: {'learning_rate': 0.02086271874273327, 'max_depth': 9, 'max_leaf_nodes': 37, 'min_samples_leaf': 181, 'l2_regularization': 0.008821821732849257, 'max_iter': 649}. Best is trial 0 with value: 0.7349282607532083.
Best trial: 0. Best value: 0.734928:  16%|█▋        | 13/80 [03:37<22:13, 19.90s/it]
[I 2025-11-16 10:52:22,351] Trial 12 finished with value: 0.7316882305458157 and parameters: {'learning_rate': 0.03499782789804734, 'max_depth': 10, 'max_leaf_nodes': 51, 'min_samples_leaf': 231, 'l2_regularization': 0.18421204205325709, 'max_iter': 799}. Best is trial 0 with value: 0.7349282607532083.
Best trial: 0. Best value: 0.734928:  18%|█▊        | 14/80 [03:51<19:44, 17.95s/it]
[I 2025-11-16 10:52:35,784] Trial 13 finished with value: 0.7299147522566856 and parameters: {'learning_rate': 0.03467439993059735, 'max_depth': 12, 'max_leaf_nodes': 36, 'min_samples_leaf': 158, 'l2_regularization': 0.46870521888784217, 'max_iter': 293}. Best is trial 0 with value: 0.7349282607532083.
Best trial: 14. Best value: 0.735714:  19%|█▉        | 15/80 [04:13<20:49, 19.22s/it]
[I 2025-11-16 10:52:57,963] Trial 14 finished with value: 0.735713815689625 and parameters: {'learning_rate': 0.04285386802917547, 'max_depth': 7, 'max_leaf_nodes': 53, 'min_samples_leaf': 98, 'l2_regularization': 0.1875326314333673, 'max_iter': 607}. Best is trial 14 with value: 0.735713815689625.
Best trial: 14. Best value: 0.735714:  20%|██        | 16/80 [04:35<21:37, 20.27s/it]
[I 2025-11-16 10:53:20,678] Trial 15 finished with value: 0.7343472270995767 and parameters: {'learning_rate': 0.04929619560339468, 'max_depth': 6, 'max_leaf_nodes': 54, 'min_samples_leaf': 98, 'l2_regularization': 0.4105096557968322, 'max_iter': 611}. Best is trial 14 with value: 0.735713815689625.
Best trial: 14. Best value: 0.735714:  21%|██▏       | 17/80 [04:43<17:23, 16.57s/it]
[I 2025-11-16 10:53:28,626] Trial 16 finished with value: 0.7340451342544652 and parameters: {'learning_rate': 0.14352151774288754, 'max_depth': 7, 'max_leaf_nodes': 56, 'min_samples_leaf': 21, 'l2_regularization': 0.19235987858839565, 'max_iter': 345}. Best is trial 14 with value: 0.735713815689625.
Best trial: 14. Best value: 0.735714:  22%|██▎       | 18/80 [05:06<18:58, 18.36s/it]
[I 2025-11-16 10:53:51,162] Trial 17 finished with value: 0.731105518452895 and parameters: {'learning_rate': 0.032745930125006865, 'max_depth': 7, 'max_leaf_nodes': 32, 'min_samples_leaf': 149, 'l2_regularization': 0.7866278717699985, 'max_iter': 583}. Best is trial 14 with value: 0.735713815689625.
Best trial: 18. Best value: 0.736395:  24%|██▍       | 19/80 [05:24<18:40, 18.37s/it]
[I 2025-11-16 10:54:09,571] Trial 18 finished with value: 0.7363947590692644 and parameters: {'learning_rate': 0.051111949452761436, 'max_depth': 8, 'max_leaf_nodes': 45, 'min_samples_leaf': 89, 'l2_regularization': 0.3731079905833916, 'max_iter': 435}. Best is trial 18 with value: 0.7363947590692644.
Best trial: 19. Best value: 0.736399:  25%|██▌       | 20/80 [05:43<18:18, 18.31s/it]
[I 2025-11-16 10:54:27,725] Trial 19 finished with value: 0.736399397491344 and parameters: {'learning_rate': 0.04333911028422001, 'max_depth': 8, 'max_leaf_nodes': 42, 'min_samples_leaf': 93, 'l2_regularization': 0.3918215140326464, 'max_iter': 422}. Best is trial 19 with value: 0.736399397491344.
Best trial: 19. Best value: 0.736399:  26%|██▋       | 21/80 [05:52<15:25, 15.68s/it]
[I 2025-11-16 10:54:37,284] Trial 20 finished with value: 0.7363720485229555 and parameters: {'learning_rate': 0.14128750789152664, 'max_depth': 9, 'max_leaf_nodes': 42, 'min_samples_leaf': 81, 'l2_regularization': 0.5859682255690067, 'max_iter': 446}. Best is trial 19 with value: 0.736399397491344.
Best trial: 19. Best value: 0.736399:  28%|██▊       | 22/80 [06:00<13:01, 13.47s/it]
[I 2025-11-16 10:54:45,584] Trial 21 finished with value: 0.7356309578527942 and parameters: {'learning_rate': 0.14032141261819722, 'max_depth': 9, 'max_leaf_nodes': 43, 'min_samples_leaf': 80, 'l2_regularization': 0.5906998166492715, 'max_iter': 435}. Best is trial 19 with value: 0.736399397491344.
Best trial: 19. Best value: 0.736399:  29%|██▉       | 23/80 [06:08<10:59, 11.58s/it]
[I 2025-11-16 10:54:52,756] Trial 22 finished with value: 0.7347970859553101 and parameters: {'learning_rate': 0.14400616186006104, 'max_depth': 8, 'max_leaf_nodes': 40, 'min_samples_leaf': 38, 'l2_regularization': 0.6973115184187372, 'max_iter': 450}. Best is trial 19 with value: 0.736399397491344.
Best trial: 23. Best value: 0.737645:  30%|███       | 24/80 [06:21<11:21, 12.17s/it]
[I 2025-11-16 10:55:06,296] Trial 23 finished with value: 0.7376450781936795 and parameters: {'learning_rate': 0.08203586194724151, 'max_depth': 9, 'max_leaf_nodes': 32, 'min_samples_leaf': 84, 'l2_regularization': 0.42410288111920663, 'max_iter': 364}. Best is trial 23 with value: 0.7376450781936795.
Best trial: 23. Best value: 0.737645:  31%|███▏      | 25/80 [06:34<11:24, 12.45s/it]
[I 2025-11-16 10:55:19,414] Trial 24 finished with value: 0.735289571918847 and parameters: {'learning_rate': 0.07751520478751042, 'max_depth': 8, 'max_leaf_nodes': 31, 'min_samples_leaf': 111, 'l2_regularization': 0.38573628923458136, 'max_iter': 358}. Best is trial 23 with value: 0.7376450781936795.
Best trial: 23. Best value: 0.737645:  32%|███▎      | 26/80 [06:47<11:12, 12.45s/it]
[I 2025-11-16 10:55:31,847] Trial 25 finished with value: 0.7302932734280189 and parameters: {'learning_rate': 0.060410528235987235, 'max_depth': 6, 'max_leaf_nodes': 34, 'min_samples_leaf': 124, 'l2_regularization': 0.48890233079421197, 'max_iter': 331}. Best is trial 23 with value: 0.7376450781936795.
Best trial: 23. Best value: 0.737645:  34%|███▍      | 27/80 [07:12<14:28, 16.38s/it]
[I 2025-11-16 10:55:57,398] Trial 26 finished with value: 0.7375834293048765 and parameters: {'learning_rate': 0.027655038501416013, 'max_depth': 10, 'max_leaf_nodes': 47, 'min_samples_leaf': 87, 'l2_regularization': 0.38508027917843346, 'max_iter': 548}. Best is trial 23 with value: 0.7376450781936795.
Best trial: 23. Best value: 0.737645:  35%|███▌      | 28/80 [07:33<15:18, 17.67s/it]
[I 2025-11-16 10:56:18,071] Trial 27 finished with value: 0.7346070482751959 and parameters: {'learning_rate': 0.02879850505419627, 'max_depth': 10, 'max_leaf_nodes': 28, 'min_samples_leaf': 41, 'l2_regularization': 0.4571341050836966, 'max_iter': 557}. Best is trial 23 with value: 0.7376450781936795.
Best trial: 23. Best value: 0.737645:  36%|███▋      | 29/80 [07:54<15:58, 18.80s/it]
[I 2025-11-16 10:56:39,498] Trial 28 finished with value: 0.7331781328077219 and parameters: {'learning_rate': 0.02736772868340478, 'max_depth': 11, 'max_leaf_nodes': 36, 'min_samples_leaf': 127, 'l2_regularization': 0.259001222815074, 'max_iter': 488}. Best is trial 23 with value: 0.7376450781936795.
Best trial: 23. Best value: 0.737645:  38%|███▊      | 30/80 [08:09<14:36, 17.54s/it]
[I 2025-11-16 10:56:54,104] Trial 29 finished with value: 0.7360477837814612 and parameters: {'learning_rate': 0.03838170732836446, 'max_depth': 11, 'max_leaf_nodes': 47, 'min_samples_leaf': 72, 'l2_regularization': 0.7470875994109165, 'max_iter': 290}. Best is trial 23 with value: 0.7376450781936795.
Best trial: 23. Best value: 0.737645:  39%|███▉      | 31/80 [08:26<14:12, 17.41s/it]
[I 2025-11-16 10:57:11,206] Trial 30 finished with value: 0.7347115808310246 and parameters: {'learning_rate': 0.027354088187797856, 'max_depth': 9, 'max_leaf_nodes': 39, 'min_samples_leaf': 51, 'l2_regularization': 0.5122476405724861, 'max_iter': 384}. Best is trial 23 with value: 0.7376450781936795.
Best trial: 23. Best value: 0.737645:  40%|████      | 32/80 [08:43<13:56, 17.42s/it]
[I 2025-11-16 10:57:28,649] Trial 31 finished with value: 0.7366030997263298 and parameters: {'learning_rate': 0.05627252464616557, 'max_depth': 8, 'max_leaf_nodes': 48, 'min_samples_leaf': 92, 'l2_regularization': 0.3760818260631519, 'max_iter': 559}. Best is trial 23 with value: 0.7376450781936795.
Best trial: 23. Best value: 0.737645:  41%|████▏     | 33/80 [09:02<13:49, 17.65s/it]
[I 2025-11-16 10:57:46,828] Trial 32 finished with value: 0.7373817941125296 and parameters: {'learning_rate': 0.05829756313762731, 'max_depth': 10, 'max_leaf_nodes': 49, 'min_samples_leaf': 107, 'l2_regularization': 0.3605665669438113, 'max_iter': 557}. Best is trial 23 with value: 0.7376450781936795.
Best trial: 23. Best value: 0.737645:  42%|████▎     | 34/80 [09:16<12:51, 16.76s/it]
[I 2025-11-16 10:58:01,534] Trial 33 finished with value: 0.7365419321871498 and parameters: {'learning_rate': 0.0800512112972542, 'max_depth': 10, 'max_leaf_nodes': 48, 'min_samples_leaf': 115, 'l2_regularization': 0.35105918132935443, 'max_iter': 550}. Best is trial 23 with value: 0.7376450781936795.
Best trial: 34. Best value: 0.739271:  44%|████▍     | 35/80 [09:36<13:11, 17.59s/it]
[I 2025-11-16 10:58:21,041] Trial 34 finished with value: 0.7392706531657476 and parameters: {'learning_rate': 0.05688837985441402, 'max_depth': 11, 'max_leaf_nodes': 50, 'min_samples_leaf': 103, 'l2_regularization': 0.24200108625133634, 'max_iter': 658}. Best is trial 34 with value: 0.7392706531657476.
Best trial: 34. Best value: 0.739271:  45%|████▌     | 36/80 [09:46<11:18, 15.41s/it]
[I 2025-11-16 10:58:31,385] Trial 35 finished with value: 0.7348849347750288 and parameters: {'learning_rate': 0.09974041039243799, 'max_depth': 11, 'max_leaf_nodes': 55, 'min_samples_leaf': 139, 'l2_regularization': 0.25577033014713, 'max_iter': 683}. Best is trial 34 with value: 0.7392706531657476.
Best trial: 34. Best value: 0.739271:  46%|████▋     | 37/80 [10:01<10:54, 15.22s/it]
[I 2025-11-16 10:58:46,158] Trial 36 finished with value: 0.7391852191939486 and parameters: {'learning_rate': 0.06883296630308035, 'max_depth': 11, 'max_leaf_nodes': 51, 'min_samples_leaf': 67, 'l2_regularization': 0.10335278222667138, 'max_iter': 725}. Best is trial 34 with value: 0.7392706531657476.
Best trial: 37. Best value: 0.740159:  48%|████▊     | 38/80 [10:18<10:58, 15.69s/it]
[I 2025-11-16 10:59:02,940] Trial 37 finished with value: 0.7401586760886502 and parameters: {'learning_rate': 0.07154794054556995, 'max_depth': 11, 'max_leaf_nodes': 58, 'min_samples_leaf': 62, 'l2_regularization': 0.08982520274920455, 'max_iter': 718}. Best is trial 37 with value: 0.7401586760886502.
Best trial: 38. Best value: 0.74052:  49%|████▉     | 39/80 [10:33<10:42, 15.67s/it] 
[I 2025-11-16 10:59:18,560] Trial 38 finished with value: 0.7405204072203996 and parameters: {'learning_rate': 0.07149121032731086, 'max_depth': 11, 'max_leaf_nodes': 60, 'min_samples_leaf': 58, 'l2_regularization': 0.1011910700746324, 'max_iter': 753}. Best is trial 38 with value: 0.7405204072203996.
Best trial: 38. Best value: 0.74052:  50%|█████     | 40/80 [10:49<10:21, 15.53s/it]
[I 2025-11-16 10:59:33,756] Trial 39 finished with value: 0.7400733873965181 and parameters: {'learning_rate': 0.0690640213160481, 'max_depth': 11, 'max_leaf_nodes': 62, 'min_samples_leaf': 22, 'l2_regularization': 0.09402932243432816, 'max_iter': 743}. Best is trial 38 with value: 0.7405204072203996.
Best trial: 40. Best value: 0.741041:  51%|█████▏    | 41/80 [11:03<09:51, 15.17s/it]
[I 2025-11-16 10:59:48,101] Trial 40 finished with value: 0.741040669720805 and parameters: {'learning_rate': 0.06897677567676878, 'max_depth': 12, 'max_leaf_nodes': 63, 'min_samples_leaf': 25, 'l2_regularization': 0.06987049159385453, 'max_iter': 757}. Best is trial 40 with value: 0.741040669720805.
Best trial: 40. Best value: 0.741041:  52%|█████▎    | 42/80 [11:17<09:26, 14.90s/it]
[I 2025-11-16 11:00:02,349] Trial 41 finished with value: 0.7408430225956941 and parameters: {'learning_rate': 0.0681495969063272, 'max_depth': 12, 'max_leaf_nodes': 63, 'min_samples_leaf': 23, 'l2_regularization': 0.10108804862171236, 'max_iter': 757}. Best is trial 40 with value: 0.741040669720805.
Best trial: 42. Best value: 0.741931:  54%|█████▍    | 43/80 [11:32<09:13, 14.96s/it]
[I 2025-11-16 11:00:17,464] Trial 42 finished with value: 0.7419307447490476 and parameters: {'learning_rate': 0.06841803908188542, 'max_depth': 12, 'max_leaf_nodes': 63, 'min_samples_leaf': 21, 'l2_regularization': 0.07425327801350062, 'max_iter': 765}. Best is trial 42 with value: 0.7419307447490476.
Best trial: 42. Best value: 0.741931:  55%|█████▌    | 44/80 [11:40<07:45, 12.93s/it]
[I 2025-11-16 11:00:25,664] Trial 43 finished with value: 0.7383279306425312 and parameters: {'learning_rate': 0.12183106703892839, 'max_depth': 12, 'max_leaf_nodes': 59, 'min_samples_leaf': 34, 'l2_regularization': 0.054285801330895944, 'max_iter': 766}. Best is trial 42 with value: 0.7419307447490476.
Best trial: 42. Best value: 0.741931:  56%|█████▋    | 45/80 [11:55<07:44, 13.28s/it]
[I 2025-11-16 11:00:39,756] Trial 44 finished with value: 0.7393268296251636 and parameters: {'learning_rate': 0.09084446846229517, 'max_depth': 12, 'max_leaf_nodes': 59, 'min_samples_leaf': 53, 'l2_regularization': 0.12817082255011028, 'max_iter': 714}. Best is trial 42 with value: 0.7419307447490476.
Best trial: 42. Best value: 0.741931:  57%|█████▊    | 46/80 [12:09<07:39, 13.52s/it]
[I 2025-11-16 11:00:53,828] Trial 45 finished with value: 0.7413201737519451 and parameters: {'learning_rate': 0.06979780621274521, 'max_depth': 12, 'max_leaf_nodes': 63, 'min_samples_leaf': 31, 'l2_regularization': 0.06213972156981895, 'max_iter': 774}. Best is trial 42 with value: 0.7419307447490476.
Best trial: 42. Best value: 0.741931:  59%|█████▉    | 47/80 [12:20<07:00, 12.74s/it]
[I 2025-11-16 11:01:04,764] Trial 46 finished with value: 0.7399411001878733 and parameters: {'learning_rate': 0.09805481724114216, 'max_depth': 12, 'max_leaf_nodes': 62, 'min_samples_leaf': 28, 'l2_regularization': 0.03513100010970292, 'max_iter': 775}. Best is trial 42 with value: 0.7419307447490476.
Best trial: 42. Best value: 0.741931:  60%|██████    | 48/80 [12:39<07:53, 14.81s/it]
[I 2025-11-16 11:01:24,398] Trial 47 finished with value: 0.7414037775164969 and parameters: {'learning_rate': 0.0642738226723709, 'max_depth': 12, 'max_leaf_nodes': 63, 'min_samples_leaf': 45, 'l2_regularization': 0.15785006809223573, 'max_iter': 764}. Best is trial 42 with value: 0.7419307447490476.
Best trial: 42. Best value: 0.741931:  61%|██████▏   | 49/80 [12:54<07:40, 14.85s/it]
[I 2025-11-16 11:01:39,330] Trial 48 finished with value: 0.6917020286505331 and parameters: {'learning_rate': 0.12021762098986997, 'max_depth': 2, 'max_leaf_nodes': 63, 'min_samples_leaf': 45, 'l2_regularization': 0.15785483574666204, 'max_iter': 792}. Best is trial 42 with value: 0.7419307447490476.
Best trial: 42. Best value: 0.741931:  62%|██████▎   | 50/80 [13:12<07:52, 15.73s/it]
[I 2025-11-16 11:01:57,136] Trial 49 finished with value: 0.7410755183805482 and parameters: {'learning_rate': 0.049747330033399643, 'max_depth': 12, 'max_leaf_nodes': 57, 'min_samples_leaf': 30, 'l2_regularization': 0.006851377787217866, 'max_iter': 685}. Best is trial 42 with value: 0.7419307447490476.
Best trial: 42. Best value: 0.741931:  64%|██████▍   | 51/80 [13:32<08:10, 16.92s/it]
[I 2025-11-16 11:02:16,832] Trial 50 finished with value: 0.740887564188667 and parameters: {'learning_rate': 0.04979110079318206, 'max_depth': 12, 'max_leaf_nodes': 57, 'min_samples_leaf': 34, 'l2_regularization': 0.0014681218449402941, 'max_iter': 681}. Best is trial 42 with value: 0.7419307447490476.
Best trial: 42. Best value: 0.741931:  65%|██████▌   | 52/80 [13:50<08:08, 17.45s/it]
[I 2025-11-16 11:02:35,513] Trial 51 finished with value: 0.7407955738210369 and parameters: {'learning_rate': 0.04671225238555334, 'max_depth': 12, 'max_leaf_nodes': 57, 'min_samples_leaf': 33, 'l2_regularization': 0.015269645597247493, 'max_iter': 691}. Best is trial 42 with value: 0.7419307447490476.
Best trial: 42. Best value: 0.741931:  66%|██████▋   | 53/80 [14:08<07:53, 17.53s/it]
[I 2025-11-16 11:02:53,231] Trial 52 finished with value: 0.7413685050438906 and parameters: {'learning_rate': 0.05407877282125691, 'max_depth': 12, 'max_leaf_nodes': 60, 'min_samples_leaf': 44, 'l2_regularization': 0.0022349123557724865, 'max_iter': 649}. Best is trial 42 with value: 0.7419307447490476.
Best trial: 42. Best value: 0.741931:  68%|██████▊   | 54/80 [14:26<07:41, 17.76s/it]
[I 2025-11-16 11:03:11,533] Trial 53 finished with value: 0.7413015455446643 and parameters: {'learning_rate': 0.06219883438466489, 'max_depth': 12, 'max_leaf_nodes': 61, 'min_samples_leaf': 46, 'l2_regularization': 0.06534469563122362, 'max_iter': 641}. Best is trial 42 with value: 0.7419307447490476.
Best trial: 42. Best value: 0.741931:  69%|██████▉   | 55/80 [14:51<08:14, 19.80s/it]
[I 2025-11-16 11:03:36,081] Trial 54 finished with value: 0.7417994179860101 and parameters: {'learning_rate': 0.038601550700063235, 'max_depth': 12, 'max_leaf_nodes': 60, 'min_samples_leaf': 45, 'l2_regularization': 0.13730774742012342, 'max_iter': 635}. Best is trial 42 with value: 0.7419307447490476.
Best trial: 42. Best value: 0.741931:  70%|███████   | 56/80 [15:14<08:21, 20.89s/it]
[I 2025-11-16 11:03:59,516] Trial 55 finished with value: 0.7413193524540981 and parameters: {'learning_rate': 0.03962963258669422, 'max_depth': 11, 'max_leaf_nodes': 61, 'min_samples_leaf': 48, 'l2_regularization': 0.16096305475103148, 'max_iter': 643}. Best is trial 42 with value: 0.7419307447490476.
Best trial: 42. Best value: 0.741931:  71%|███████▏  | 57/80 [15:40<08:36, 22.45s/it]
[I 2025-11-16 11:04:25,607] Trial 56 finished with value: 0.7406418873571055 and parameters: {'learning_rate': 0.0385416424542653, 'max_depth': 12, 'max_leaf_nodes': 53, 'min_samples_leaf': 74, 'l2_regularization': 0.15408164673298716, 'max_iter': 623}. Best is trial 42 with value: 0.7419307447490476.
Best trial: 42. Best value: 0.741931:  72%|███████▎  | 58/80 [16:05<08:30, 23.19s/it]
[I 2025-11-16 11:04:50,534] Trial 57 finished with value: 0.7396053401693022 and parameters: {'learning_rate': 0.03840967998421716, 'max_depth': 10, 'max_leaf_nodes': 60, 'min_samples_leaf': 57, 'l2_regularization': 0.19943477358079972, 'max_iter': 588}. Best is trial 42 with value: 0.7419307447490476.
Best trial: 42. Best value: 0.741931:  74%|███████▍  | 59/80 [16:20<07:12, 20.61s/it]
[I 2025-11-16 11:05:05,115] Trial 58 finished with value: 0.7319848501880354 and parameters: {'learning_rate': 0.05307955340183216, 'max_depth': 11, 'max_leaf_nodes': 16, 'min_samples_leaf': 46, 'l2_regularization': 0.1406369027376661, 'max_iter': 659}. Best is trial 42 with value: 0.7419307447490476.
Best trial: 42. Best value: 0.741931:  75%|███████▌  | 60/80 [16:58<08:34, 25.72s/it]
[I 2025-11-16 11:05:42,773] Trial 59 finished with value: 0.7352704752464689 and parameters: {'learning_rate': 0.03213777497915103, 'max_depth': 11, 'max_leaf_nodes': 55, 'min_samples_leaf': 200, 'l2_regularization': 0.22007980807152006, 'max_iter': 704}. Best is trial 42 with value: 0.7419307447490476.
Best trial: 42. Best value: 0.741931:  76%|███████▋  | 61/80 [17:13<07:11, 22.71s/it]
[I 2025-11-16 11:05:58,447] Trial 60 finished with value: 0.7173309455899367 and parameters: {'learning_rate': 0.043312269774130455, 'max_depth': 4, 'max_leaf_nodes': 61, 'min_samples_leaf': 255, 'l2_regularization': 0.2920630412153761, 'max_iter': 632}. Best is trial 42 with value: 0.7419307447490476.
Best trial: 42. Best value: 0.741931:  78%|███████▊  | 62/80 [17:30<06:17, 20.96s/it]
[I 2025-11-16 11:06:15,332] Trial 61 finished with value: 0.7408793535222433 and parameters: {'learning_rate': 0.06238794816588139, 'max_depth': 12, 'max_leaf_nodes': 61, 'min_samples_leaf': 46, 'l2_regularization': 0.055774398708802464, 'max_iter': 800}. Best is trial 42 with value: 0.7419307447490476.
Best trial: 42. Best value: 0.741931:  79%|███████▉  | 63/80 [17:59<06:34, 23.20s/it]
[I 2025-11-16 11:06:43,746] Trial 62 finished with value: 0.7396583591324075 and parameters: {'learning_rate': 0.08524132917368282, 'max_depth': 12, 'max_leaf_nodes': 60, 'min_samples_leaf': 39, 'l2_regularization': 0.17108989576674896, 'max_iter': 639}. Best is trial 42 with value: 0.7419307447490476.
Best trial: 42. Best value: 0.741931:  80%|████████  | 64/80 [35:00<1:26:04, 322.80s/it]
[I 2025-11-16 11:23:45,633] Trial 63 finished with value: 0.7416460928382413 and parameters: {'learning_rate': 0.06208196357278812, 'max_depth': 12, 'max_leaf_nodes': 58, 'min_samples_leaf': 51, 'l2_regularization': 0.04904074463795199, 'max_iter': 596}. Best is trial 42 with value: 0.7419307447490476.
Best trial: 42. Best value: 0.741931:  81%|████████▏ | 65/80 [35:08<57:01, 228.10s/it]  
[I 2025-11-16 11:23:52,763] Trial 64 finished with value: 0.7300557603068649 and parameters: {'learning_rate': 0.2537188628514156, 'max_depth': 11, 'max_leaf_nodes': 58, 'min_samples_leaf': 66, 'l2_regularization': 0.128863635418127, 'max_iter': 599}. Best is trial 42 with value: 0.7419307447490476.
Best trial: 65. Best value: 0.742633:  82%|████████▎ | 66/80 [35:46<39:56, 171.14s/it]
[I 2025-11-16 11:24:31,004] Trial 65 finished with value: 0.7426327378200158 and parameters: {'learning_rate': 0.04073650623789187, 'max_depth': 12, 'max_leaf_nodes': 63, 'min_samples_leaf': 52, 'l2_regularization': 0.039499597468768, 'max_iter': 524}. Best is trial 65 with value: 0.7426327378200158.
Best trial: 65. Best value: 0.742633:  84%|████████▍ | 67/80 [36:20<28:11, 130.09s/it]
[I 2025-11-16 11:25:05,287] Trial 66 finished with value: 0.7414024037748506 and parameters: {'learning_rate': 0.04609873404717625, 'max_depth': 12, 'max_leaf_nodes': 63, 'min_samples_leaf': 56, 'l2_regularization': 0.027060874453631742, 'max_iter': 523}. Best is trial 65 with value: 0.7426327378200158.
Best trial: 65. Best value: 0.742633:  85%|████████▌ | 68/80 [36:51<20:05, 100.42s/it]
[I 2025-11-16 11:25:36,494] Trial 67 finished with value: 0.7397157717169857 and parameters: {'learning_rate': 0.04592038233434139, 'max_depth': 12, 'max_leaf_nodes': 55, 'min_samples_leaf': 76, 'l2_regularization': 0.04185046227258746, 'max_iter': 482}. Best is trial 65 with value: 0.7426327378200158.
Best trial: 65. Best value: 0.742633:  86%|████████▋ | 69/80 [37:29<14:58, 81.66s/it] 
[I 2025-11-16 11:26:14,366] Trial 68 finished with value: 0.7376124164331423 and parameters: {'learning_rate': 0.023301777228200676, 'max_depth': 10, 'max_leaf_nodes': 58, 'min_samples_leaf': 56, 'l2_regularization': 0.023166425871723382, 'max_iter': 474}. Best is trial 65 with value: 0.7426327378200158.
Best trial: 65. Best value: 0.742633:  88%|████████▊ | 70/80 [38:05<11:19, 67.97s/it]
[I 2025-11-16 11:26:50,396] Trial 69 finished with value: 0.741851879950254 and parameters: {'learning_rate': 0.03491095377940972, 'max_depth': 12, 'max_leaf_nodes': 59, 'min_samples_leaf': 20, 'l2_regularization': 0.22227340658481973, 'max_iter': 527}. Best is trial 65 with value: 0.7426327378200158.
Best trial: 65. Best value: 0.742633:  89%|████████▉ | 71/80 [38:39<08:38, 57.60s/it]
[I 2025-11-16 11:27:23,787] Trial 70 finished with value: 0.7406304914236497 and parameters: {'learning_rate': 0.03445432435714846, 'max_depth': 11, 'max_leaf_nodes': 52, 'min_samples_leaf': 21, 'l2_regularization': 0.21284469916389392, 'max_iter': 526}. Best is trial 65 with value: 0.7426327378200158.
Best trial: 65. Best value: 0.742633:  90%|█████████ | 72/80 [39:06<06:29, 48.63s/it]
[I 2025-11-16 11:27:51,497] Trial 71 finished with value: 0.728065403679589 and parameters: {'learning_rate': 0.031993398944175154, 'max_depth': 5, 'max_leaf_nodes': 59, 'min_samples_leaf': 39, 'l2_regularization': 0.07767697897531209, 'max_iter': 519}. Best is trial 65 with value: 0.7426327378200158.
Best trial: 65. Best value: 0.742633:  91%|█████████▏| 73/80 [39:39<05:06, 43.82s/it]
[I 2025-11-16 11:28:24,110] Trial 72 finished with value: 0.7313898485929912 and parameters: {'learning_rate': 0.0421861407119892, 'max_depth': 12, 'max_leaf_nodes': 22, 'min_samples_leaf': 167, 'l2_regularization': 0.8616202398023846, 'max_iter': 572}. Best is trial 65 with value: 0.7426327378200158.
Best trial: 65. Best value: 0.742633:  92%|█████████▎| 74/80 [40:13<04:04, 40.83s/it]
[I 2025-11-16 11:28:57,942] Trial 73 finished with value: 0.7419918433978544 and parameters: {'learning_rate': 0.05461950119510961, 'max_depth': 12, 'max_leaf_nodes': 56, 'min_samples_leaf': 65, 'l2_regularization': 0.11543177357435402, 'max_iter': 505}. Best is trial 65 with value: 0.7426327378200158.
Best trial: 65. Best value: 0.742633:  94%|█████████▍| 75/80 [40:49<03:17, 39.51s/it]
[I 2025-11-16 11:29:34,376] Trial 74 finished with value: 0.7413419524343794 and parameters: {'learning_rate': 0.04053896920127856, 'max_depth': 11, 'max_leaf_nodes': 56, 'min_samples_leaf': 62, 'l2_regularization': 0.11292611577241626, 'max_iter': 533}. Best is trial 65 with value: 0.7426327378200158.
Best trial: 65. Best value: 0.742633:  95%|█████████▌| 76/80 [41:27<02:36, 39.00s/it]
[I 2025-11-16 11:30:12,199] Trial 75 finished with value: 0.7407715722332817 and parameters: {'learning_rate': 0.030307274016765147, 'max_depth': 12, 'max_leaf_nodes': 62, 'min_samples_leaf': 52, 'l2_regularization': 0.30241955180790375, 'max_iter': 506}. Best is trial 65 with value: 0.7426327378200158.
Best trial: 65. Best value: 0.742633:  96%|█████████▋| 77/80 [42:04<01:54, 38.33s/it]
[I 2025-11-16 11:30:48,958] Trial 76 finished with value: 0.7404414476932603 and parameters: {'learning_rate': 0.03679945760506036, 'max_depth': 11, 'max_leaf_nodes': 54, 'min_samples_leaf': 70, 'l2_regularization': 0.13174170709951938, 'max_iter': 533}. Best is trial 65 with value: 0.7426327378200158.
Best trial: 65. Best value: 0.742633:  98%|█████████▊| 78/80 [42:26<01:06, 33.47s/it]
[I 2025-11-16 11:31:11,088] Trial 77 finished with value: 0.7421389824130944 and parameters: {'learning_rate': 0.07499857813010635, 'max_depth': 12, 'max_leaf_nodes': 63, 'min_samples_leaf': 38, 'l2_regularization': 0.1880064864151393, 'max_iter': 501}. Best is trial 65 with value: 0.7426327378200158.
Best trial: 65. Best value: 0.742633:  99%|█████████▉| 79/80 [43:02<00:34, 34.29s/it]
[I 2025-11-16 11:31:47,297] Trial 78 finished with value: 0.7391095725817763 and parameters: {'learning_rate': 0.0250805804342493, 'max_depth': 11, 'max_leaf_nodes': 58, 'min_samples_leaf': 38, 'l2_regularization': 0.2352315357260482, 'max_iter': 465}. Best is trial 65 with value: 0.7426327378200158.
Best trial: 65. Best value: 0.742633: 100%|██████████| 80/80 [43:22<00:00, 32.54s/it]
[I 2025-11-16 11:32:07,595] Trial 79 finished with value: 0.740185779986132 and parameters: {'learning_rate': 0.07446284951992652, 'max_depth': 10, 'max_leaf_nodes': 56, 'min_samples_leaf': 25, 'l2_regularization': 0.1866495939622716, 'max_iter': 495}. Best is trial 65 with value: 0.7426327378200158.

[GB] Best CV R²: 0.74263
[GB] Best params: {'learning_rate': 0.04073650623789187, 'max_depth': 12, 'max_leaf_nodes': 63, 'min_samples_leaf': 52, 'l2_regularization': 0.039499597468768, 'max_iter': 524}

[Final 5-fold CV] Tuned model performance:
RandomForest R²: 0.71408 ± 0.00590
HistGradientBoosting R²: 0.74263 ± 0.00627

=== Best Params Summary ===
[RandomForest] {'n_estimators': 528, 'max_depth': 21, 'min_samples_split': 8, 'min_samples_leaf': 4, 'max_features': 0.4839384174656995}
[HistGradientBoosting] {'learning_rate': 0.04073650623789187, 'max_depth': 12, 'max_leaf_nodes': 63, 'min_samples_leaf': 52, 'l2_regularization': 0.039499597468768, 'max_iter': 524}

Tuned models are available as: best_rf, best_gb'''

'''
[Final 5-fold CV] Tuned model performance:
RandomForest R²: 0.71408 ± 0.00590
HistGradientBoosting R²: 0.74263 ± 0.00627

=== Best Params Summary ===
[RandomForest] {'n_estimators': 528, 'max_depth': 21, 'min_samples_split': 8, 'min_samples_leaf': 4, 'max_features': 0.4839384174656995}
[HistGradientBoosting] {'learning_rate': 0.04073650623789187, 'max_depth': 12, 'max_leaf_nodes': 63, 'min_samples_leaf': 52, 'l2_regularization': 0.039499597468768, 'max_iter': 524}

Tuned models are available as: best_rf, best_gb
'''

"\n[Final 5-fold CV] Tuned model performance:\nRandomForest R²: 0.71408 ± 0.00590\nHistGradientBoosting R²: 0.74263 ± 0.00627\n\n=== Best Params Summary ===\n[RandomForest] {'n_estimators': 528, 'max_depth': 21, 'min_samples_split': 8, 'min_samples_leaf': 4, 'max_features': 0.4839384174656995}\n[HistGradientBoosting] {'learning_rate': 0.04073650623789187, 'max_depth': 12, 'max_leaf_nodes': 63, 'min_samples_leaf': 52, 'l2_regularization': 0.039499597468768, 'max_iter': 524}\n\nTuned models are available as: best_rf, best_gb\n"

In [ ]:
import json

# Save best params to JSON
with open("rf_best_params_v2.json", "w") as f:
    json.dump(rf_study.best_params, f, indent=4)

with open("gb_best_params_v2.json", "w") as f:
    json.dump(gb_study.best_params, f, indent=4)

print("Saved best params to:")
print("  rf_best_params_v2.json")
print("  gb_best_params_v2.json")

Saved best params to:
  rf_best_params_v2.json
  gb_best_params_v2.json
